Pull total number of orders that were completed on 18th March 2023

Pull total number of orders that were completed on 18th March 2023 with the first name ‘John’ and last name Doe’

Pull total number of customers that purchased in January 2023 and the average amount spend per customer

Pull the departments that generated less than $600 in 2022

What is the most and least revenue we have generated by an order

What were the orders that were purchased in our most lucrative order

TABLE INFO :

SALES – Date, Order_id, Item_id, Customer_id, Quantity, Revenue

ITEMS – Item_id, Item_name, price, department

CUSTOMERS- customer_id, first_name,last_name,Address

Small Dataset can be generated using chatgpt.

In [45]:
import pandas as pd
import sqlite3

customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4],
    'first_name': ['John', 'Alice', 'Bob', 'John'],
    'last_name': ['Doe', 'Smith', 'Johnson', 'Snow'],
    'address': ['NY', 'CA', 'TX', 'WA']
})
items = pd.DataFrame({
    'item_id': [101, 102, 103, 104],
    'item_name': ['T-Shirt', 'Laptop', 'Book', 'Mug'],
    'price': [20, 900, 15, 10],
    'department': ['Apparel', 'Electronics', 'Books', 'Kitchenware']
})
sales = pd.DataFrame({
    'date': ['2023-03-18', '2023-03-18', '2023-03-18', '2023-01-10', '2023-01-22', '2022-11-11', '2022-07-05'],
    'order_id': [5001, 5002, 5003, 5004, 5005, 5006, 5007],
    'item_id': [101, 103, 102, 104, 101, 104, 103],
    'customer_id': [1, 2, 1, 3, 2, 4, 3],
    'quantity': [2, 1, 1, 2, 1, 1, 5],
    'revenue': [40, 15, 900, 20, 20, 10, 75]
})

# 创建内存数据库并加载数据
conn = sqlite3.connect(":memory:")
customers.to_sql('CUSTOMERS', conn, index=False, if_exists='replace')
items.to_sql('ITEMS', conn, index=False, if_exists='replace')
sales.to_sql('SALES', conn, index=False, if_exists='replace')

# 查询
q1 = pd.read_sql_query("SELECT COUNT(DISTINCT order_id) AS total_orders FROM SALES WHERE date = '2023-03-18'", conn)
q2 = pd.read_sql_query("""
    SELECT COUNT(DISTINCT s.order_id) AS john_doe_orders 
    FROM SALES s JOIN CUSTOMERS c ON s.customer_id = c.customer_id 
    WHERE s.date = '2023-03-18' AND c.first_name = 'John' AND c.last_name = 'Doe'
""", conn)
q3 = pd.read_sql_query("""
    SELECT COUNT(DISTINCT t.customer_id) AS total_customers, 
           ROUND(AVG(t.total_spend), 2) AS avg_spent_per_customer 
    FROM (SELECT customer_id, SUM(revenue) AS total_spend 
          FROM SALES WHERE date BETWEEN '2023-01-01' AND '2023-01-31' 
          GROUP BY customer_id) t
""", conn)
q4 = pd.read_sql_query("""
    SELECT i.department, SUM(s.revenue) AS total_revenue 
    FROM SALES s JOIN ITEMS i ON s.item_id = i.item_id 
    WHERE strftime('%Y', s.date) = '2022' 
    GROUP BY i.department HAVING total_revenue < 600
""", conn)
q5 = pd.read_sql_query("""
    SELECT MAX(order_revenue) AS max_revenue, MIN(order_revenue) AS min_revenue 
    FROM (SELECT order_id, SUM(revenue) AS order_revenue FROM SALES GROUP BY order_id) t
""", conn)
q6 = pd.read_sql_query("""
    WITH OrderRevenues AS (SELECT order_id, SUM(revenue) AS total_revenue FROM SALES GROUP BY order_id)
    SELECT s.order_id, s.date, i.item_name, s.quantity, s.revenue 
    FROM SALES s JOIN ITEMS i ON s.item_id = i.item_id 
    WHERE s.order_id = (SELECT order_id FROM OrderRevenues ORDER BY total_revenue DESC LIMIT 1)
""", conn)

# 输出结果
print("1. Orders on 2023-03-18:\n", q1)
print("\n2. Orders on 2023-03-18 by John Doe:\n", q2)
print("\n3. Jan 2023 customers and avg spend:\n", q3)
print("\n4. Departments with < $600 in 2022:\n", q4)
print("\n5. Max and Min Revenue by Order:\n", q5)
print("\n6. Most Lucrative Order Details:\n", q6)

# 关闭连接（可选）
conn.close()

1. Orders on 2023-03-18:
    total_orders
0             3

2. Orders on 2023-03-18 by John Doe:
    john_doe_orders
0                2

3. Jan 2023 customers and avg spend:
    total_customers  avg_spent_per_customer
0                2                    20.0

4. Departments with < $600 in 2022:
     department  total_revenue
0        Books             75
1  Kitchenware             10

5. Max and Min Revenue by Order:
    max_revenue  min_revenue
0          900           10

6. Most Lucrative Order Details:
    order_id        date item_name  quantity  revenue
0      5003  2023-03-18    Laptop         1      900
